<a href="https://colab.research.google.com/github/l43lu/NLP/blob/master/Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 2.x
import pandas as pd
import re
import tensorflow as tf
import os
import io
tf.__version__

'2.6.0'

In [2]:
# Download the zip file
path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip", origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip", extract=True)

212992/203415 [===============================] - 0s 2us/step


In [3]:
# Unzip the file into a folder
!unzip $path_to_zip -d data

Archive:  /root/.keras/datasets/smsspamcollection.zip
  inflating: data/SMSSpamCollection  
  inflating: data/readme             


In [4]:
# Let's see if we read the data correctly
lines = io.open('data/SMSSpamCollection').read().strip().split('\n')
lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [5]:
spam_dataset = []
for line in lines:
  label, text = line.split('\t')
  if label.strip() == 'spam':
    spam_dataset.append((1, text.strip()))
  else:
    spam_dataset.append(((0, text.strip())))
  

print(spam_dataset[0])


(0, 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')


In [7]:
df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])

In [11]:
df.head()

,Spam,Message,Capitals,Punctuation,Length
0,0,"Go until jurong point, crazy.. Available only ...",3,28,111
1,0,Ok lar... Joking wif u oni...,2,11,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,10,33,155
3,0,U dun say so early hor... U c already then say...,2,16,49
4,0,"Nah I don't think he goes to usf, he lives aro...",2,14,61


In [9]:
def message_length(x):
  # returns total number of characters
  return len(x)

def num_capitals(x):
  _, count = re.subn(r'[A-Z]', '', x) # only works in english
  return count

def num_punctuation(x):
  _, count = re.subn(r'\W', '', x)
  return count

In [10]:
df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df['Message'].apply(message_length)
df.describe()

,Spam,Capitals,Punctuation,Length
count,5574.000000,5574.000000,5574.000000,5574.000000
mean,0.134015,5.621636,18.942591,80.443488
std,0.340699,11.683233,14.825994,59.841746
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


The following code can be used to split the dataset into training and test sets, with 80% of the records in the training set and the rest in the test set. Further more, labels will be removed from both the training and test sets:

In [13]:
train = df.sample(frac=0.8,random_state=42)
test  = df.drop(train.index)

x_train = train[['Length', 'Capitals', 'Punctuation']]
y_train = train[['Spam']]

x_test = test[['Length', 'Capitals', 'Punctuation']]
y_test = test[['Spam']]

In [15]:
# A function is defined that allows the construction of models with different numbers of inputs and hidden units

# This model uses binary cross-entropy for computing loss and the Adam optimizer for training. The key metric, given that this is a binary classification problem, 
# is accuracy. The default parameters passed to the function are sufficient as only three features are being passed in.

# Basic 1-layer neural network model for evaluation
def make_model(input_dims=3, num_units=12):
  model = tf.keras.Sequential()
  # Adds a densely-connected layer with 12 units to the model:
  model.add(tf.keras.layers.Dense(num_units, input_dim=input_dims, activation='relu'))
  # Add a sigmoid layer with a binary output unit:
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [16]:
# train our simple baseline model with only three features
model = make_model()
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 1ms/step - loss: 1.1455 - accuracy: 0.8569
Epoch 2/10
446/446 [==============================] - 1s 1ms/step - loss: 0.3109 - accuracy: 0.9112
Epoch 3/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2515 - accuracy: 0.9208
Epoch 4/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2164 - accuracy: 0.9300
Epoch 5/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2103 - accuracy: 0.9291
Epoch 6/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2057 - accuracy: 0.9309
Epoch 7/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2036 - accuracy: 0.9285
Epoch 8/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2003 - accuracy: 0.9336
Epoch 9/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2003 - accuracy: 0.9329
Epoch 10/10
446/446 [==============================] - 1s 1ms/step - loss: 0.1987 - accuracy: 0.9352

In [17]:
# Evaluating it
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 1000us/step - loss: 0.2046 - accuracy: 0.9363


[0.20455998182296753, 0.9363228678703308]

In [20]:
#A quick verification can be performed by plotting the confusion matrix to see the performance
y_train_pred = model.predict(x_train)
tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3866,    1],
       [ 592,    0]], dtype=int32)>